In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)



In [3]:
def arima_forecast_district(series, district_name, seasonal=False, max_p=5, max_q=5):
    """
    Complete ARIMA forecasting with data storage and plot saving
    """
    # Create directory structure
    os.makedirs('ARIMA', exist_ok=True)
    
    # 1. Stationarity Check
    adf_result = adfuller(series.dropna())
    is_stationary = adf_result[1] < 0.05

    # 2. Train-Test Split (80:20)
    train_size = int(len(series) * 0.8)
    train, test = series.iloc[:train_size], series.iloc[train_size:]

    # 3. Auto-ARIMA Modeling
    model = auto_arima(
        train,
        start_p=0, start_q=0,
        max_p=max_p, max_q=max_q,
        d=None,
        seasonal=seasonal,
        trace=False,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    best_order = model.order

    # 4. Model Fitting
    arima_model = ARIMA(train, order=best_order)
    model_fit = arima_model.fit()
    
    # 5. Forecasting with Confidence Intervals
    forecast_result = model_fit.get_forecast(steps=len(test))
    forecast = forecast_result.predicted_mean
    conf_int = forecast_result.conf_int()

    # 6. Create forecast DataFrame
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': test.index,
        'actual': test.values,
        'forecast': forecast.values,
        'lower_ci': conf_int.iloc[:, 0],
        'upper_ci': conf_int.iloc[:, 1]
    })

    # 7. Save forecasts to CSV (append mode)
    forecast_csv_path = os.path.join('ARIMA', 'arima_forecasts.csv')
    write_header = not os.path.exists(forecast_csv_path)
    forecast_df.to_csv(forecast_csv_path, 
                      mode='a', 
                      header=write_header, 
                      index=False)

    # 8. Calculate and save metrics
    rmse = np.sqrt(mean_squared_error(test, forecast))
    metrics_df = pd.DataFrame([{
        'district': district_name,
        'rmse': rmse,
        'best_order': str(best_order),
        'stationary': is_stationary
    }])

    metrics_csv_path = os.path.join('ARIMA', 'arima_results.csv')
    write_header_metrics = not os.path.exists(metrics_csv_path)
    metrics_df.to_csv(metrics_csv_path, 
                     mode='a', 
                     header=write_header_metrics, 
                     index=False)

    # 9. Save plot
    plt.figure(figsize=(12, 6))
    plt.plot(train, label='Training Data')
    plt.plot(test, label='Actual Values', color='navy')
    plt.plot(forecast, label='Forecast', color='darkorange')
    plt.fill_between(forecast.index, 
                    conf_int.iloc[:, 0],
                    conf_int.iloc[:, 1],
                    color='orange', alpha=0.1)
    plt.title(f'ARIMA{best_order} Forecast for {district_name}\nRMSE: {rmse:.2f}')
    plt.legend()
    plt.savefig(os.path.join('ARIMA', f'ARIMA_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df
    }

In [4]:
districts = data['district'].unique()
rmse_values = []
districts

array(['AHMEDNAGAR', 'AKOLA', 'AMRAVATI', 'AURANGABAD', 'BEED',
       'BHANDARA', 'BULDHANA', 'CHANDRAPUR', 'DHULE', 'GADCHIROLI',
       'GONDIA', 'HINGOLI', 'JALGAON', 'JALNA', 'KOLHAPUR', 'LATUR',
       'MUMBAI', 'NAGPUR', 'NANDED', 'NANDURBAR', 'NASHIK', 'OSMANABAD',
       'PALGHAR', 'PARBHANI', 'PUNE', 'RAIGAD', 'RATNAGIRI', 'SANGLI',
       'SATARA', 'SINDHUDURG', 'SOLAPUR', 'THANE', 'WARDHA', 'WASHIM',
       'YAVATMAL', 'MUMBAI SUBURBAN'], dtype=object)

In [5]:

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I8"].asfreq('MS')
        
        results = arima_forecast_district(ts, district)
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district        rmse best_order  stationary
0  AHMEDNAGAR  944.375065  (0, 1, 1)       False

=== Forecast Data ===
              district       date  actual     forecast     lower_ci  \
2020-06-01  AHMEDNAGAR 2020-06-01    6372  6843.879401  5741.934102   
2020-07-01  AHMEDNAGAR 2020-07-01    5982  6843.879401  5507.037068   
2020-08-01  AHMEDNAGAR 2020-08-01    5559  6843.879401  5307.646540   
2020-09-01  AHMEDNAGAR 2020-09-01    5903  6843.879401  5131.315421   
2020-10-01  AHMEDNAGAR 2020-10-01    5719  6843.879401  4971.517426   

               upper_ci  
2020-06-01  7945.824700  
2020-07-01  8180.721734  
2020-08-01  8380.112262  
2020-09-01  8556.443380  
2020-10-01  8716.241375  
=== Metrics ===
  district        rmse best_order  stationary
0    AKOLA  214.859256  (0, 1, 0)       False

=== Forecast Data ===
           district       date  actual  forecast     lower_ci     upper_ci
2020-06-01    AKOLA 2020-06-01    1575    1709.0  1389.510399  2028.489601

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


=== Metrics ===
  district        rmse best_order  stationary
0     PUNE  920.330078  (3, 0, 2)        True

=== Forecast Data ===
           district       date  actual     forecast     lower_ci     upper_ci
2020-06-01     PUNE 2020-06-01    6054  5701.233045  4765.793798  6636.672291
2020-07-01     PUNE 2020-07-01    5520  6547.057199  5259.412246  7834.702152
2020-08-01     PUNE 2020-08-01    5630  6785.727288  5491.094374  8080.360202
2020-09-01     PUNE 2020-09-01    5491  6953.701934  5605.359208  8302.044660
2020-10-01     PUNE 2020-10-01    6135  6969.363893  5542.226356  8396.501431
=== Metrics ===
  district         rmse best_order  stationary
0   RAIGAD  1093.112937  (0, 1, 1)       False

=== Forecast Data ===
           district       date  actual     forecast    lower_ci     upper_ci
2020-06-01   RAIGAD 2020-06-01    1582  1806.704038  892.741687  2720.666389
2020-07-01   RAIGAD 2020-07-01    1563  1806.704038  720.171942  2893.236134
2020-08-01   RAIGAD 2020-08-01    151

{'district': 'MUMBAI SUBURBAN',
 'forecast_df':                    district       date  actual     forecast     lower_ci  \
 2021-01-01  MUMBAI SUBURBAN 2021-01-01    9489  9097.559482  6780.596372   
 2021-02-01  MUMBAI SUBURBAN 2021-02-01    9380  8734.534763  5717.987712   
 2021-03-01  MUMBAI SUBURBAN 2021-03-01    9076  8431.882755  5012.387881   
 
                 upper_ci  
 2021-01-01  11414.522592  
 2021-02-01  11751.081813  
 2021-03-01  11851.377630  ,
 'metrics_df':           district        rmse best_order  stationary
 0  MUMBAI SUBURBAN  572.927592  (1, 0, 0)       False}

In [6]:
pd.DataFrame(rmse_values)

""
